In [ ]:
code_bucket_name = "banpu-interns-data-exchange"
lambda_function_name = "invoke-model-endpoint"
endpoint_name = "tensorflow-inference-2024-07-11-09-00-17-984"

target_bucket_name = "banpu-interns-data-exchange"
target_bucket_prefix = "data/"

In [ ]:
import boto3

# Use AWS CLI named profile
session = boto3.Session(profile_name="bivlab")
lambda_client = session.client("lambda")
iam_client = session.client("iam")
s3_client = session.client("s3")
sts_client = session.client("sts")

account_id = sts_client.get_caller_identity()["Account"]
current_region = session.region_name

In [ ]:
import json

# Create a role for the Lambda function
role_name = "LambdaSageMakerRole"

assume_role_policy_document = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "lambda.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}

try:
    create_role = iam_client.create_role(
      RoleName=role_name,
      AssumeRolePolicyDocument=json.dumps(assume_role_policy_document)
    )

    role_arn = create_role["Role"]["Arn"]
except iam_client.exceptions.EntityAlreadyExistsException:
    role_arn = iam_client.get_role(RoleName=role_name)["Role"]["Arn"]

# Attach policies to the role
policy_arns = [
  "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess",
  "arn:aws:iam::aws:policy/AmazonSageMakerFullAccess"
]

for policy_arn in policy_arns:
  try:
    iam_client.attach_role_policy(
      RoleName=role_name,
      PolicyArn=policy_arn
    )
  except iam_client.exceptions.PolicyAlreadyExistsException:
    pass

In [ ]:
import os
import zipfile

zip_file_name = lambda_function_name + ".zip"

# Upload source code to S3 bucket
with zipfile.ZipFile(zip_file_name, "w") as f:
    f.write("handler.py")

s3_client.upload_file(zip_file_name, code_bucket_name, zip_file_name)
os.remove(zip_file_name)

In [ ]:
# Deploy AWS Lambda Function to invoke model from S3 bucket events
environment_variables = {"Variables": {"ENDPOINT_NAME": endpoint_name}}

# Delete if Lambda function exists
try:
    lambda_client.delete_function(FunctionName=lambda_function_name)
except lambda_client.exceptions.ResourceNotFoundException:
    pass

# Create the Lambda function
lambda_function = lambda_client.create_function(
    FunctionName=lambda_function_name,
    Runtime="python3.11",
    Role=role_arn,
    Handler="handler.lambda_handler",
    Environment=environment_variables,
    Code={
        "S3Bucket": code_bucket_name,
        "S3Key": zip_file_name,
    },
    Layers=[
        "arn:aws:lambda:ap-southeast-1:770693421928:layer:Klayers-p311-numpy:7",
        "arn:aws:lambda:ap-southeast-1:770693421928:layer:Klayers-p311-Pillow:4",
    ],
    Description="Lambda function to invoke Sagemaker model endpoint from S3 bucket events",
    Timeout=600,  # Timeout in seconds
    MemorySize=512,  # Memory in MB
)

In [ ]:
# Add permission to lambda for S3 invocation
try:
  lambda_client.add_permission(
      FunctionName=lambda_function_name,
      StatementId="s3-invoke",
      Action="lambda:InvokeFunction",
      Principal="s3.amazonaws.com",
      SourceArn=f"arn:aws:s3:::{target_bucket_name}",
      SourceAccount=account_id,
  )
except lambda_client.exceptions.ResourceConflictException:
  pass

# Bind S3 bucket events to the Lambda function
s3_client.put_bucket_notification_configuration(
  Bucket=target_bucket_name,
  NotificationConfiguration={
    "LambdaFunctionConfigurations": [
      {
        "LambdaFunctionArn": lambda_function["FunctionArn"],
        "Events": ["s3:ObjectCreated:*"],
        "Filter": {
          "Key": {
            "FilterRules": [
              {
                "Name": "prefix",
                "Value": target_bucket_prefix
              }
            ]
          }
        }
      }
    ]
  }
)